<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [90]:
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings('ignore')

In [91]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [92]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT,
    connect_timeout = 1800 # пытался увеличить время до дисконнекта, но это почему то не работало
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [93]:
# текст запроса
query_3_1 = f'''SELECT
                    COUNT(*)
                FROM public.vacancies
'''
pd.read_sql_query(query_3_1, connection)

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [94]:
# текст запроса
query = f'''SELECT
                COUNT(*)
            FROM public.employers
'''
pd.read_sql_query(query, connection)

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [95]:
# текст запроса
query = f'''SELECT
                COUNT(*)
            FROM public.areas
'''
pd.read_sql_query(query, connection)

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [96]:
# текст запроса
query = f'''SELECT
                COUNT(*)
            FROM public.industries
'''
pd.read_sql_query(query, connection)

,count
0,294


***

### выводы по предварительному анализу данных
В базе довольно много вакансий, работодателей, регинов и индустрий

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [97]:
# текст запроса
query = f'''SELECT
                a.name,
                count(v.id)
            FROM public.vacancies v
            JOIN public.areas a on v.area_id = a.id
            GROUP by 1
            ORDER by 2 desc
'''
pd.read_sql_query(query, connection)

,name,count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [98]:
# текст запроса
query = f'''SELECT
                count(v.id)
            FROM public.vacancies v
            WHERE salary_from is not NULL or salary_to is not NULL
'''
pd.read_sql_query(query, connection)

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [99]:
# текст запроса
query = f'''SELECT
                round(avg(v.salary_from),0),
                round(avg(v.salary_to),0)
            FROM public.vacancies v
            WHERE salary_from is not NULL or salary_to is not NULL
'''
pd.read_sql_query(query, connection)

,round,round
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [100]:
# текст запроса
query = f'''SELECT
                schedule,
                employment,
                count(v.id)
            FROM public.vacancies v
            GROUP by 1,2
            ORDER by 3 desc
'''
pd.read_sql_query(query, connection)

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [102]:
# текст запроса
query = f'''SELECT
                experience
            FROM public.vacancies v
            GROUP by 1
            ORDER by count(v.id)
'''
pd.read_sql_query(query, connection)

***

### выводы по детальному анализу вакансий
1. Больше всего вакансий доступно в столицах и густонаселенных городах
2. Только в половине вакансий указана хотя бы одна из зарплат
3. Среднее по вилке зарплат: 71065 - 110537 
4. Наиболее рапсространены вакансии с полной занятостью (графики Полный день - Удаленная работа - Гибкий график)
5. Чаще ищут работников хотя бы с минимальным опытом работы (1-3 года). Опыт работы более 6 лет наименее востребован - реже вакансий совсем без опыта

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [135]:
# текст запроса
query = f'''SELECT
                e.name,
                count(v.id)
            FROM public.vacancies v
            JOIN public.employers e on e.id = v.employer_id
            GROUP by 1
            ORDER by 2 desc
'''
pd.read_sql_query(query, connection)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331
...,...,...
14761,им. Тьюринга,1
14762,LLC Novalab Tech,1
14763,Компания Мир упаковки,1
14764,Лифинцев Олег Иванович,1


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [136]:
# текст запроса
query = f'''SELECT
                a.name,
                count(distinct e.id) число_работодателей,
                count(v.id) число_вакансий
            FROM public.areas a
            LEFT JOIN public.employers e on a.id = e.area
            LEFT JOIN public.vacancies v on a.id = v.area_id and v.employer_id = e.id
            GROUP by 1
            ORDER by 3, 2 desc
'''
pd.read_sql_query(query, connection)

,name,число_работодателей,число_вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Великобритания,23,0
4,Эстония,19,0
...,...,...,...
1357,Екатеринбург,609,960
1358,Алматы,721,1069
1359,Санкт-Петербург,2217,1811
1360,Минск,1115,1858


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [137]:
# текст запроса
query = f'''SELECT
                e.name,
                count(distinct v.area_id) количество_регионов
            FROM public.employers e 
            JOIN public.vacancies v on v.employer_id = e.id
            GROUP by 1
            ORDER by 2 desc
'''
pd.read_sql_query(query, connection)

,name,количество_регионов
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [110]:
# текст запроса
query = f'''SELECT
                count(*)
            FROM public.employers e 
            LEFT JOIN public.employers_industries ei on ei.employer_id = e.id
            WHERE industry_id is null
'''
pd.read_sql_query(query, connection)

,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [112]:
# текст запроса
query = f'''SELECT
                e.name
            FROM public.employers e 
            JOIN public.employers_industries ei on ei.employer_id = e.id
            GROUP by e.name
            HAVING count(industry_id) = 4
            order by 1
'''
pd.read_sql_query(query, connection)

,name
0,101 Интернет
1,21vek.by
2,2ГИС
3,2К
4,4 пикселя +
...,...
1131,ЮРИОН
1132,ЮТИП Технологии
1133,ЯКласс
1134,ЯрНео


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [138]:
# текст запроса
query = f'''SELECT
                count(*)
            FROM public.employers_industries ei
            JOIN public.industries i on ei.industry_id = i.id
            WHERE name = 'Разработка программного обеспечения'
'''
pd.read_sql_query(query, connection)

,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [116]:
# код для получения списка городов-милионников
html = pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')
city_list = list(html[1].iloc[:,1])
sql_city_list = ', '.join(f"'{city}'" for city in city_list)
sql_city_list = sql_city_list.replace('[a]','')

sql_city_list

"'Москва', 'Санкт-Петербург', 'Новосибирск', 'Екатеринбург', 'Казань', 'Нижний Новгород', 'Челябинск', 'Красноярск', 'Самара', 'Уфа', 'Ростов-на-Дону', 'Омск', 'Краснодар', 'Воронеж', 'Пермь', 'Волгоград'"

In [117]:
# текст запроса
query = f'''WITH t as
            (SELECT
                a.name,
                count(v.id)::int cnt
            FROM public.areas a
            JOIN public.vacancies v on a.id = v.area_id
            JOIN public.employers e on v.employer_id = e.id
            WHERE e.name = 'Яндекс' and a.name in ({sql_city_list})
            GROUP by 1
            ORDER by 2 desc)
            
            SELECT
                *
            FROM t
            
            UNION all
            
            SELECT
                'Total',
                sum(cnt)
            FROM t           
'''
pd.read_sql_query(query, connection)


,name,cnt
0,Москва,54
1,Санкт-Петербург,42
2,Екатеринбург,39
3,Нижний Новгород,36
4,Новосибирск,35
5,Воронеж,32
6,Краснодар,30
7,Самара,26
8,Уфа,26
9,Казань,25


***

### выводы по анализу работодателей
1. Больше всего вакансий от IT-компаний и банков. У Яндекса очень много вакансий
2. Среди регионов с компаниями без вакансий преобладают просто самые большие - страны.
3. Кроме очевидных Яндекса и Ростелекома всплывают внезапные, вроде "Поляков Денис Иванович".
4. У примерно четверти работодателей не указана сфера деятельности (либо наша база не полная).
5. У примерно 5% работодателей указано 4 сферы деятельности.
6. У примерно 15% работодателей среди сфер деятельности указана разработка программного обеспечения.
7. У Яндекса 485 вакансий в городах-миллионниках.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [118]:
# текст запроса
query = f'''SELECT
                count(*)
            FROM public.vacancies v
            WHERE lower(name) LIKE '%data%' or lower(name) like '%данн%'
'''
pd.read_sql_query(query, connection)

,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [119]:
#сохраняю условие на будущее
ds_check = '''(name ILIKE ANY (array['%data scientist%', '%data science%','%исследователь данных%', '%machine learning%', '%машинн%обучен%'])
            or (name like '%ML%' and name not ilike '%HTML%'))
'''
# Вопрос проверяющему: почему при условии 
# ILIKE ANY ('%data scientist%', '%data science%', ...)
# SQL выдает syntax error с указанием на запятую и работет только через array?

In [120]:
query = f'''select 
               count(*)
            from vacancies as v
            Where {ds_check} and 
            (name ILIKE '%junior%'
            or experience = 'Нет опыта'
            or employment = 'Cтажировка')
'''
pd.read_sql_query(query, connection)

,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [121]:
query = f'''select 
               count(*)
            from vacancies as v
            Where {ds_check} and
            (key_skills iLIKE '%postgres%'
            or key_skills LIKE '%SQL%')
            
'''
pd.read_sql_query(query, connection)

,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [122]:
query = f'''select 
               count(*)
            from vacancies as v
            Where {ds_check} and
            key_skills LIKE '%Python%'
            
'''
pd.read_sql_query(query, connection)

,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [123]:
query = f'''SELECT 
               avg(LENGTH(key_skills) - length(replace(key_skills, '\t','')) + 1)
            from public.vacancies v
            Where {ds_check} and key_skills is not null
            
'''
pd.read_sql_query(query, connection)
# и опять SQL выдаёт ошибку на запятую если написать 
# length(replace(key_skills, char(9),''), что не так?

,avg
0,6.406032


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [124]:
query = f'''SELECT 
                experience,
                round(avg(salary))
            FROM (
                    SELECT
                        experience, 
                        salary_from as salary
                    from public.vacancies v
                    Where {ds_check} and salary_from is not null and salary_to is null
                    
                    Union all
                    
                    SELECT
                        experience, 
                        salary_to
                    from public.vacancies v
                    Where {ds_check} and salary_to is not null and salary_from is null
                    
                    Union all
                    
                    SELECT
                        experience, 
                        (salary_to + salary_from)/2
                    from public.vacancies v
                    Where {ds_check} and salary_to is not null and salary_from is not null
                ) as foo
            GROUP by 1
'''
pd.read_sql_query(query, connection)

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

### выводы по предметному анализу
1. Вакансии, связанные с данными составляют около 3,5% от всех вакансий.
2. При строгом поиске вакансий DS всего 480, из них 51 для начинающих. 
3. Навыки SQL и PostgreSQL требутся для 42% DS-вакансий
4. Знание Python требуется для 73% DS-вакансий
5. В среднем для DS-вакансий требуется 6-7 навыков
6. Средняя зарплата DS-специалиста в зависмиости от опыта работы: Нет опыта-74643.0, От 1 года до 3 лет - 139675.0, От 3 до 6 лет - 243115.0
   В DS-вакансиях нет требований к опыту > 6 лет. Видимо, отрасль еще молодая

# Общий вывод по проекту

1. Больше всего вакансий от IT-компаний и банков
2. Рынок труда намного обширнее в густонаселенных регионах
3. Интересно сравнить DS-вакансии с остальными (Лучше именно IT, но это сложнее)

In [125]:
import plotly.express as px

# Дополнительный анализ

## Посмотрим разброс вакансий по индустриям

In [140]:
query = f'''SELECT
                i.name,
                count (v.id)   
            FROM vacancies v
            JOIN employers e on v.employer_id = e.id
            JOIN (
                SELECT 
                    DISTINCT employer_id,
                    industry_id   
                FROM employers_industries
                ) as short_ei
                
                on short_ei.employer_id = e.id
            
            JOIN industries i on short_ei.industry_id = i.id
            GROUP by 1
            Order by 2 desc
            LIMIT 10
'''
df = pd.read_sql_query(query, connection)

In [141]:
fig = px.pie(
    df,
    names = 'name',
    values = 'count',
    title = 'Топ-10 индустрий, представленных в вакансиях'
)
fig.update_layout(showlegend=True, legend=dict(
    title_font_family='Courier New',
    font=dict(
        size=8
    )
))
fig.show()

Беглый анализ по индустриям представленным в вакансиях *(1 работодатель - 1 индустрия и отображение только топ-10 индустрий для упрощения запроса)* показывает, что значительную долю рынка занимают: "Разработка программного обеспечения", "Системная интеграция,  автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг", "Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее)" и "Банк"


## Оценим основные навыки, требуемые для DS-вакансий

In [142]:
query = f'''select 
               *
            from vacancies as v
            Where {ds_check}
'''
df = pd.read_sql_query(query, connection)

In [143]:
from collections import Counter
ds_skills=[]
for line in df['key_skills']:
    if line is not None:
        for skill in line.split('\t'):
            ds_skills.append((skill).upper()) # upper отлавливает часть разных написаний, но не все
skill_dict = dict(Counter(ds_skills))
skill_dict['MACHINE LEARNING'] += skill_dict['ML'] # MACHINE LEARNING и ML дублируются в топе 
skill_dict.pop('ML')
sorted_skill_dict = sorted(skill_dict.items(), key=lambda x:x[1],  reverse = True)

fig_data = pd.DataFrame(sorted_skill_dict[:10])

fig = px.histogram(
    fig_data,
    x = 0,
    y = 1,
    color = 0,
    title = 'Топ-10 навыков, востребованных в DS-вакансиях'
)
fig.show()


Основы двух основных навыков получены, готовимся учить ML :)

## Сравним зарплаты DS-вакансий и остальных

In [145]:
query = f'''SELECT
                
                experience,
                round(avg(salary)),
                'ds_vacancies' as  type
            FROM (
                    SELECT
                        experience, 
                        salary_from as salary
                    from public.vacancies v
                    Where {ds_check} and salary_from is not null and salary_to is null
                    
                    Union all
                    
                    SELECT
                        experience, 
                        salary_to
                    from public.vacancies v
                    Where {ds_check} and salary_to is not null and salary_from is null
                    
                    Union all
                    
                    SELECT
                        experience, 
                        (salary_to + salary_from)/2
                    from public.vacancies v
                    Where {ds_check} and salary_to is not null and salary_from is not null
                ) as foo
            GROUP by 1
            
            UNION ALL
            
            SELECT 
                experience,
                round(avg(salary)),
                'all_vacancies' as type
            FROM (
                    SELECT
                        experience, 
                        salary_from as salary
                    from public.vacancies v
                    Where salary_from is not null and salary_to is null
                    
                    Union all
                    
                    SELECT
                        experience, 
                        salary_to
                    from public.vacancies v
                    Where salary_to is not null and salary_from is null
                    
                    Union all
                    
                    SELECT
                        experience, 
                        (salary_to + salary_from)/2
                    from public.vacancies v
                    Where salary_to is not null and salary_from is not null
                ) as foo
            GROUP by 1
            
            
'''
df = pd.read_sql_query(query, connection)


In [146]:
new_df = df.pivot(index='type', columns='experience', values = 'round')

cols = new_df.columns.tolist() #организуем опыт по порядку возрастания
cols = cols[1:] + cols[:1]
new_df = new_df[cols]
fig = px.imshow(
    new_df
)
fig.show()

В сравнении с остальными вакансиями, зарплата на DS-вакансиях значительно выше
*(лучше бы сравнить с IT-вакансиями, но это значительно дольше вытаскивать)* 